In [29]:
import csv
import re
import sys
import time
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import metrics, tree
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

In [30]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str, encoding = 'utf-8',
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])
#df = shuffle(df)

In [31]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
#     punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
#     no_punct = ""
#     for char in sample:
#         if char not in punctuations:
#             no_punct = no_punct + char
#     return no_punct
    return re.sub(r'[^\w\s\&\#\@\$\%\_]','',sample)

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au')] #and not bool(re.search(r'\d',word))]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = [w for w in sample.split(' ') if len(w) >= 2]
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = [w for w in sample.split(' ') if len(w) >= 2]
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()

def lemmy(sample):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    words = [w for w in sample.split(' ') if len(w) >= 2]
    lemmed_text = ""
    for word in words:
        lemmed_text = lemmed_text + lemmatizer.lemmatize(word) + " "
    return lemmed_text.rstrip()
    
def snowball(sample):
    words = [w for w in sample.split(' ') if len(w) >= 2]
    stemmer = SnowballStemmer("english")
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + stemmer.stem(word) + " "
    return stemmed_text.rstrip()

def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
    sample = porter_stem(sample)
    return sample


In [32]:
""" Data creation """
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)

In [33]:
X_train_, X_test_, y_train, y_test = train_test_split(text_data, Y, test_size=0.25, shuffle=False)

In [34]:
# try to use sklearn stop_words later
# 711, 0.688
# 1178, 0.978
# max_features=818, ngram_range=(1, 2), min_df = 0
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=578, ngram_range=(1, 3), min_df = 5)
X_train = count.fit_transform(X_train_).toarray()
X_test = count.transform(X_test_).toarray()
# print(count.get_feature_names())
# size = len(count.vocabulary_)
print(len(count.vocabulary_))

578


In [35]:
clf = MultinomialNB(alpha = 0.7546, fit_prior = True)
#clf = BernoulliNB(alpha = 1.0, fit_prior = False)
model = clf.fit(X_train, y_train)
#model = clf.fit(X, Y)
#print(model.class_log_prior_ )
# text_clf_red = Pipeline([('vect', CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer)), 
#                        ('reducer', SelectKBest(chi2, k=200)),
#                        ('clf', MultinomialNB())
#                        ])
# model_new = text_clf_red.fit(text_data[:1500],Y[:1500])
# model_new = text_clf_red.fit(text_data,Y)

In [36]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10000       0.52      0.61      0.56        56
       10001       0.39      0.25      0.31        36
       10002       0.55      0.58      0.56        31
       10003       0.33      0.53      0.41        87
       10004       0.00      0.00      0.00         2
       10005       0.69      0.63      0.66        52
       10006       0.44      0.41      0.42        44
       10007       0.00      0.00      0.00         2
       10008       0.60      0.70      0.65        46
       10009       0.00      0.00      0.00         4
       10010       0.27      0.27      0.27        11
       10011       0.00      0.00      0.00         7
       10012       0.00      0.00      0.00         4
       10013       0.62      0.41      0.49        37
       10014       0.00      0.00      0.00         6
       10015       0.47      0.58      0.52        24
       10016       0.29      0.14      0.19        14
       10017       0.17    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [37]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

       10000       0.80      0.79      0.79       188
       10001       0.71      0.67      0.69       104
       10002       0.69      0.83      0.76        99
       10003       0.69      0.69      0.69       271
       10004       1.00      0.60      0.75        15
       10005       0.72      0.74      0.73       142
       10006       0.75      0.74      0.75       145
       10007       0.00      0.00      0.00         5
       10008       0.77      0.81      0.79       117
       10009       0.75      0.75      0.75        12
       10010       0.54      0.73      0.62        45
       10011       1.00      0.17      0.29         6
       10012       0.75      0.57      0.65        21
       10013       0.73      0.87      0.79        67
       10014       0.91      0.43      0.59        23
       10015       0.78      0.92      0.84        95
       10016       0.64      0.60      0.62        45
       10017       0.77    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
clf.score(X_train,y_train)

0.7313333333333333

In [39]:
clf.score(X_test,y_test)

0.462

In [40]:
# y_pred = model_new.predict(text_data[1500:])
# print(classification_report(Y[1500:], y_pred))

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
clf = make_pipeline(CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=578, ngram_range=(1, 3), min_df = 5), MultinomialNB(alpha = 0.7546))
scores = cross_val_score(clf,text_data,Y,cv=6,scoring = 'f1_micro')
print(scores)
print("F1 micro Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.49562682 0.43362832 0.43113772 0.42042042 0.5        0.46749226]
F1 micro Accuracy: 0.46 (+/- 0.06)


In [42]:
# y_pred = model_new.predict(text_data[:1500])
# print(classification_report(Y[:1500], y_pred))